In [13]:
import pandas as pd 
import numpy as np 
from pickle import load 
import datetime as dt 


EARNINGS_PATH = 'earnings/earnings.pkl'
earnings = load(open(EARNINGS_PATH, 'rb'))
[earnings.pop(x) for x in ['hsbc', 'djt']]
stocks = list(earnings.keys())
print(dt.datetime.now())

2024-09-12 21:22:18.250428


In [15]:
out = []
for x in stocks:
    df = earnings[x]['earnings'].copy().reset_index()
    earn_dates = [str(x).split(':00-')[0] for x in df['Earnings Date']]
    df['Earnings Date'] = pd.to_datetime(earn_dates)
    df.insert(0, 'stock', x.upper())
    start = dt.datetime.now()
    end = start + dt.timedelta(days=15)
    outdf = df[
        (df['Earnings Date'] >= start.date().strftime('%Y-%m-%d')) & 
        (df['Earnings Date'] <= end.date().strftime('%Y-%m-%d'))
    ]
    out.append(outdf)
    
out = pd.concat(out)
report_time = np.where(out['Earnings Date'].dt.hour < 11, 'AM', 'PM')
out.insert(2, 'Hour', out['Earnings Date'].dt.hour)
out.insert(3, 'Time', report_time)
out['Earnings Date'] = out['Earnings Date'].dt.strftime('%Y-%m-%d')

out_final = out.drop_duplicates(subset = ['stock']).rename(columns = {
    'Earnings Date': 'Date',
    'EPS Estimate': 'EPS',
}).drop(columns = ['Reported EPS', 'Surprise(%)']).reset_index(drop = True)

print(out_final.shape)
out_final[out_final['Date'] >= "2024-07-25"].sort_values(['Date', 'Hour']).head(20)

(3, 5)


,stock,Date,Hour,Time,EPS
1,KR,2024-09-12,8,AM,0.91
2,PGR,2024-09-13,6,AM,NaN
0,MU,2024-09-25,16,PM,1.15


In [19]:
import sys
sys.path.append('/Users/jerald/Documents/Dir/Python/Stocks')

from signals.Options.find_plays import Play as dxp 
from bin.main import get_path 

connections = get_path('../')
find_plays = dxp(connections)
find_plays.verbose = False

Options db Connected: 2024-09-12 21:47:44.229082
Prices Connected: 2024-09-12 21:47:44.230360


In [20]:
out_final.stock.to_list()[:8]

['MU', 'KR', 'PGR']

In [21]:
from tqdm import tqdm 

earning_plays = [x.lower() for x in out_final.stock]
earning_dates = [x for x in out_final.Date]
earnings_time = [x for x in out_final.Time]
ep = list(zip(earning_plays, earning_dates, earnings_time))
plays = []
for x, exp, hr in tqdm(ep):
    tmp = find_plays.run(x)
    tmp.insert(1, 'earnings_date', exp)
    tmp.insert(2, 'hour', hr)
    plays.append(tmp)

100%|██████████| 3/3 [00:00<00:00,  6.96it/s]


In [22]:
tmp = pd.concat(plays)
tmp = tmp.rename(columns = {'impliedvolatility': 'iv', 'openinterest': 'oi',})  
tmp = tmp.drop(columns = ['gatherdate', 'flag', 'percentchange', 'pct_chg'])

In [23]:
# print(list(zip(earning_plays, earning_dates)))
j = sorted(list(tmp.earnings_date.unique()))
tmp.earnings_date = pd.to_datetime(tmp.earnings_date)
for i in j[:10]:
    stks = tmp[tmp.earnings_date == i].stock.unique()
    # print(f'Earnings: {i:%A %b %d %Y} - {list(stks)}')
    print(f'Earnings: {i} - {list(stks)}')

Earnings: 2024-09-12 - ['KR']
Earnings: 2024-09-13 - ['PGR']
Earnings: 2024-09-25 - ['MU']


In [24]:
out_final.sort_values(['Date','Time'])

,stock,Date,Hour,Time,EPS
1,KR,2024-09-12,8,AM,0.91
2,PGR,2024-09-13,6,AM,NaN
0,MU,2024-09-25,16,PM,1.15


In [26]:
from IPython.display import display
pd.set_option('display.max_columns', 500)

st = 'SNOW'
# tmp = tmp[tmp.index <= "2024-08-25"]
# Filter where the % needed to breakeven is less than 2 times the expected move 
# emp = (np.abs(tmp.be) < tmp.empct)
# print('------------- 2 sd ---------------')
# display(tmp[(tmp.stock == st) & (tmp.sd == 2) & emp].sort_index())
# print('------------- 1 sd ---------------')
# # Filter where the % needed to breakeven is less than the expected move
# emp = (np.abs(tmp.be) < tmp.empct)
# display(tmp[(tmp.stock == st) & (tmp.sd == 1) & emp].sort_index())

In [27]:
# t = tmp[['earnings_date','stock', 'type', 'sd', 'strike', 'stk_price', 'lastprice', 'volume', 'openinterest', 'voi', 'be', 'em', 'empct']].copy()
t = tmp.copy()
t['upper'] = t.stk_price + t.em
t['dup'] = t.stk_price + (t.em * 2)
t['lower'] = t.stk_price - t.em
t['dlo'] = t.stk_price - (t.em * 2)

tweets = t[['stock', 'dup','upper','stk_price','lower', 'dlo', 'empct']].sort_values('stock').drop_duplicates()
tweets

,stock,dup,upper,stk_price,lower,dlo,empct
expiry,,,,,,,
2025-06-20,KR,65.74,60.14,54.54,48.94,43.34,0.097543
2025-01-17,KR,60.64,57.59,54.54,51.49,48.44,0.053126
2025-04-17,KR,65.24,59.89,54.54,49.19,43.84,0.093188
2024-10-04,KR,57.96,56.25,54.54,52.83,51.12,0.029785
2024-09-20,KR,58.82,56.68,54.54,52.40,50.26,0.037275
2024-10-25,KR,NaN,NaN,54.54,NaN,NaN,NaN
2024-09-13,KR,57.44,55.99,54.54,53.09,51.64,0.025257
2024-09-27,KR,64.18,59.36,54.54,49.72,44.90,0.083957
2026-01-16,KR,68.58,61.56,54.54,47.52,40.50,0.122277


In [28]:
t[np.abs(t.be) < t.empct * 2].sort_values(['amnt'], ascending = False).head(20).round(2)

,stock,earnings_date,hour,type,strike,sd,contractsymbol,stk_price,lastprice,ask,bid,change,volume,oi,iv,stk_price_chg,stk_price_avg_30d,stk_price_avg_5d,lastprice_chg,lastprice_avg_30d,lastprice_avg_5d,iv_chg,iv_avg_5d,iv_avg_30d,iv_avg_all,vol_chg,oi_chg,amnt,moneyness,voi,be,em,empct,upper,dup,lower,dlo
expiry,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-13,KR,2024-09-12,AM,Call,56.0,1,KR240913C00056000,54.54,0.12,0.12,0.10,-0.15,1484,4419,0.36,3.51,52.97,52.55,-0.06,0.74,0.36,-0.44,0.60,0.40,0.40,1177.0,3617.0,3310,1.03,0.34,0.03,1.45,0.03,55.99,57.44,53.09,51.64
2024-09-27,KR,2024-09-12,AM,Call,56.0,1,KR240927C00056000,54.54,0.56,0.55,0.49,0.07,32,236,0.25,3.51,52.71,52.55,0.06,0.81,0.70,-0.18,0.39,0.33,0.33,15.0,181.0,164,1.03,0.14,0.04,4.82,0.08,59.36,64.18,49.72,44.90
2024-09-13,KR,2024-09-12,AM,Call,57.0,2,KR240913C00057000,54.54,0.04,0.05,0.03,-0.11,1880,1309,0.40,3.51,52.97,52.55,-0.06,0.55,0.24,-0.39,0.61,0.40,0.40,1471.0,564.0,155,1.05,1.44,0.05,1.45,0.03,55.99,57.44,53.09,51.64
2024-09-20,KR,2024-09-12,AM,Call,57.0,2,KR240920C00057000,54.54,0.17,0.18,0.15,-0.08,161,740,0.27,3.51,52.65,52.55,-0.04,0.43,0.35,-0.25,0.45,0.38,0.38,-2.0,299.0,136,1.05,0.22,0.05,2.14,0.04,56.68,58.82,52.40,50.26
2024-12-20,KR,2024-09-12,AM,Call,55.0,1,KR241220C00055000,54.54,3.00,2.97,2.90,1.10,139,1938,0.28,3.51,53.10,52.55,1.35,2.62,2.32,-0.02,0.30,0.28,0.28,132.0,108.0,101,1.01,0.07,0.06,2.90,0.05,57.44,60.34,51.64,48.74
2024-12-20,KR,2024-09-12,AM,Call,55.0,1,KR241220C00055000,54.54,3.00,2.97,2.90,1.10,139,1938,0.28,3.51,53.10,52.55,1.35,2.62,2.32,-0.02,0.30,0.28,0.28,132.0,108.0,101,1.01,0.07,0.06,2.90,0.05,57.44,60.34,51.64,48.74
2024-10-11,MU,2024-09-25,PM,Call,90.0,2,MU241011C00090000,87.49,5.44,5.50,5.20,-1.21,517,164,0.64,-1.40,87.37,87.37,-0.26,5.18,5.18,0.04,0.62,0.62,0.62,475.0,123.0,81,1.03,3.15,0.09,6.85,0.07,94.34,101.19,80.64,73.79
2026-01-16,MU,2024-09-25,PM,Call,90.0,1,MU260116C00090000,87.49,21.00,21.00,20.60,-1.75,463,1039,0.54,-1.40,95.80,87.69,0.01,26.37,20.54,0.01,0.54,0.55,0.49,460.0,42.0,39,1.03,0.45,0.27,16.52,0.18,104.01,120.53,70.97,54.45
2026-01-16,MU,2024-09-25,PM,Call,90.0,1,MU260116C00090000,87.49,21.00,21.00,20.60,-1.75,463,1039,0.54,-1.40,95.80,87.69,0.01,26.37,20.54,0.01,0.54,0.55,0.49,460.0,42.0,39,1.03,0.45,0.27,16.52,0.18,104.01,120.53,70.97,54.45


In [22]:
tmp['money'] = tmp['strike'] / tmp['stk_price']


In [32]:
# iv = tmp[tmp.openinterest > 1000].copy()
iv = tmp[tmp.oi_chg > 0].copy().reset_index().set_index('stock').drop(columns = ['expiry'])
iv = iv[iv.stk_price > 2]
iv.earnings_date = pd.to_datetime(iv.earnings_date, format = '%Y-%m-%d')

iv['iv_30diff'] = iv.iv_avg_30d - iv.iv
iv['iv_5diff'] = iv.iv_avg_5d - iv.iv

iv_head = iv[(iv['iv_avg_30d']<iv.iv) & iv.index.isin([x.upper() for x in earning_plays]) & (iv['lastprice']> .10)].sort_values('lastprice', ascending = True)


In [33]:
iv_head.head(30).sort_values('iv_30diff', ascending = False)

,earnings_date,hour,type,strike,sd,contractsymbol,stk_price,lastprice,change,volume,oi,iv,stk_price_chg,stk_price_avg_30d,stk_price_avg_5d,lastprice_chg,lastprice_avg_30d,lastprice_avg_5d,iv_chg,iv_avg_5d,iv_avg_30d,iv_avg_all,vol_chg,oi_chg,amnt,moneyness,voi,be,em,empct,money,iv_30diff,iv_5diff
stock,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
KR,2024-09-06,AM,Put,49.0,2,KR240927P00049000,52.62,0.50,0.00,2,5,0.309577,-0.16,52.700000,52.700000,0.00,0.500000,0.500000,0.009277,0.304939,0.304939,0.304939,0.0,2.0,0,0.931205,0.400000,-0.078297,4.25,0.076729,0.931205,-0.004639,-0.004639
AFRM,2024-08-28,PM,Put,22.0,2,AFRM240927P00022000,32.23,0.33,0.00,10,71,0.867189,0.43,30.156667,30.795000,0.00,0.461111,0.366667,-0.046875,0.885906,0.850045,0.850045,0.0,10.0,0,0.682594,0.140845,-0.327645,3.47,0.102280,0.682594,-0.017144,0.018717
LI,2024-08-28,AM,Call,23.5,1,LI240920C00023500,21.20,0.55,0.00,1,17,0.667972,0.49,21.021667,21.021667,0.00,0.626667,0.626667,-0.001953,0.644535,0.644535,0.644535,0.0,1.0,0,1.108491,0.058824,0.134434,3.20,0.143396,1.108491,-0.023437,-0.023437
KR,2024-09-06,AM,Put,49.0,2,KR240913P00049000,52.62,0.41,0.03,14,78,0.368659,-0.16,52.598333,52.328333,0.03,0.515000,0.486667,0.014160,0.339362,0.333401,0.333401,3.0,5.0,0,0.931205,0.179487,-0.076587,4.60,0.083048,0.931205,-0.035258,-0.029297
GME,2024-09-04,AM,Put,18.5,2,GME240920P00018500,20.97,0.51,0.12,8,36,0.725589,-0.59,21.265000,21.265000,0.12,0.450000,0.450000,0.071288,0.689945,0.689945,0.689945,-60.0,31.0,0,0.882213,0.222222,-0.142108,3.40,0.154030,0.882213,-0.035644,-0.035644
AFRM,2024-08-28,PM,Put,22.0,2,AFRM240920P00022000,32.23,0.21,-0.06,11,70,0.986328,0.43,30.795000,30.795000,-0.06,0.360000,0.360000,-0.000977,0.938640,0.938640,0.938640,-10.0,18.0,0,0.682594,0.157143,-0.323922,6.50,0.191592,0.682594,-0.047688,-0.047688
LI,2024-08-28,AM,Put,17.0,2,LI240920P00017000,21.20,0.17,-0.02,16,1182,0.664066,0.49,19.834000,21.021667,-0.02,0.567333,0.211667,0.015625,0.638350,0.606742,0.604397,-29.0,35.0,0,0.801887,0.013536,-0.206132,3.20,0.143396,0.801887,-0.057324,-0.025716
LI,2024-08-28,AM,Put,19.0,1,LI240920P00019000,21.20,0.52,-0.13,9,2780,0.648441,0.49,19.834000,21.021667,-0.11,1.231333,0.588333,0.032226,0.610681,0.589197,0.588046,-61.0,13.0,0,0.896226,0.003237,-0.128302,3.20,0.143396,0.896226,-0.059244,-0.037760
AFRM,2024-08-28,PM,Put,22.5,2,AFRM241018P00022500,32.23,0.51,-0.06,773,1974,0.811525,0.43,28.008667,30.795000,-0.06,1.357000,0.706667,-0.007812,0.798179,0.751956,0.755651,745.0,18.0,0,0.698107,0.391591,-0.317716,3.75,0.110534,0.698107,-0.059570,-0.013346
